In [1]:
# Play with qubits.
# Qubits will be placed on a grid.

import cirq
# define the length of the grid
length = 3
# define the qubits on the grid using list comprehension
qubits  = [cirq.GridQubit(i, j) for i in range(length) for j in range(length)]
print(qubits)
# prints:
# cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]
# essentially the total combinations of upto length 3.
# 'GridQubit' implements the 'Qid' class, which is hashable.

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


In [2]:
# Now that we have qubits, let's construct a 'Circuit' on these qubits.
# Here we will apply the Hadamard gate 'H' to every qubit, whose
# row index plu column index is even. And an 'X' gate to those with odd
# sum of row and column index.
circuit = cirq.Circuit()
circuit.append(cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0)
circuit.append(cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1)
print(circuit)
# Will print the qubits that have the Hadamard gate apllied and those with Pauli X gate applied.
# Here we are applying single qubit gates.
# Gates appearing in a single vertical line, as they are here,
# constitute a moment(collection of operations).

(0, 0): ───H───

(0, 1): ───X───

(0, 2): ───H───

(1, 0): ───X───

(1, 1): ───H───

(1, 2): ───X───

(2, 0): ───H───

(2, 1): ───X───

(2, 2): ───H───


In [3]:
# Let's change the fact that we have a "moment" for these qubits.
circuit = cirq.Circuit()
circuit.append([cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0],
               strategy=cirq.InsertStrategy.EARLIEST)
circuit.append([cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1],
               strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
print(circuit)
# Now the circuit has staggered the operations, giving us two moments.

(0, 0): ───H───────

(0, 1): ───────X───

(0, 2): ───H───────

(1, 0): ───────X───

(1, 1): ───H───────

(1, 2): ───────X───

(2, 0): ───H───────

(2, 1): ───────X───

(2, 2): ───H───────


In [4]:
for i , m in enumerate(circuit):
    print('Moment {}: {}'.format(i, m))
# This gives us the breakdown of where our qubits lie w.r.t. the moments.

Moment 0: H((0, 0)) and H((0, 2)) and H((1, 1)) and H((2, 0)) and H((2, 2))
Moment 1: X((0, 1)) and X((1, 0)) and X((1, 2)) and X((2, 1))


In [5]:
# Define a function that takes in relevant parameters 
# and then yields the operations for the sub circuit, then this
# can be appended to the Circuit
def rot_x_layer(length, half_turns):
    """Yields X rotations by half_turns on a square gridof given length."""
    rot = cirq.XPowGate(exponent=half_turns)
    for i in range(length):
        for j in range(length):
            yield rot(cirq.GridQubit(i, j))
            
circuit = cirq.Circuit()
circuit.append(rot_x_layer(2, 0.1))
print(circuit)

(0, 0): ───X^0.1───

(0, 1): ───X^0.1───

(1, 0): ───X^0.1───

(1, 1): ───X^0.1───


In [6]:
# Here we generate random problem instances
import random
def rand2d(rows, cols):
    return [[random.choice([+1, -1]) for _ in range(cols)] for _ in range(rows)]

def random_instance(length):
    # transverse field terms
    h = rand2d(length, length)
    # links within a row
    jr = rand2d(length -1, length)
    # links within a column
    jc = rand2d(length, length -1)
    return (h, jr, jc)

h, jr, jc = random_instance(3)
print('transverse fields: {}'.format(h))
print('row j fields: {}'.format(jr))
print('column j fields: {}'.format(jr))
# prints something like
# transverse fields: [[-1, 1, -1], [1, -1, -1], [-1, 1, -1]]
# row j fields: [[1, 1, -1], [1, -1, 1]]
# column j fields: [[1, -1], [-1, 1], [-1, 1]]

transverse fields: [[-1, -1, 1], [-1, -1, 1], [-1, 1, -1]]
row j fields: [[-1, 1, 1], [1, 1, 1]]
column j fields: [[-1, 1, 1], [1, 1, 1]]


In [7]:
"""
Make an ansatz which consists of one step of a circuit made up of
1. Applying an XPowGate for the same parameter for all qubits. The method written above.
2. Applying a ZPowGate for the same parameter for all qubits where the transverse field term is +1.
"""
def rot_z_layer(h, half_turns):
    """Yields Z rotations by half_turns conditioned on the field h."""
    gate = cirq.ZPowGate(exponent=half_turns)
    for i, h_row in enumerate(h):
        for j, h_ij in enumerate(h_row):
            if h_ij == 1:
                yield(gate(cirq.GridQubit(i, j)))

In [8]:
"""
Apply CZPowGate for the same parameter between all qubits where the coupling field term J
is +1. If the field is -1 apply CZPowGate conjugated by X gates on all qubits.
"""
def rot_11_layer(jr, jc, half_turns):
    """Yields rotations about |11> conditioned on the jr, jc fields."""
    gate = cirq.CZPowGate(exponent=half_turns)
    for i, jr_row in enumerate(jr):
        for j, jr_ij in enumerate(jr_row):
            if jr_ij ==-1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i + 1, j))
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i + 1, j))
            
            if jr_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i + 1, j))
                
    for i, jc_row in enumerate(jc):
        for j, jc_ij in enumerate(jc_row):
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j + 1))
            yield gate(cirq.GridQubit(i, j),
                       cirq.GridQubit(i, j + 1))
            if jc_ij == -1:
                yield cirq.X(cirq.GridQubit(i, j))
                yield cirq.X(cirq.GridQubit(i, j + 1))

In [9]:
def one_step(h, jr, jc, x_half_turns, h_half_turns, j_half_turns):
    length = len(h)
    yield rot_x_layer(length, x_half_turns)
    yield rot_z_layer(h, h_half_turns)
    yield rot_11_layer(jr, jc, j_half_turns)
    
h, jr, jc = random_instance(3)

circuit = cirq.Circuit()
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3),
               strategy=cirq.InsertStrategy.EARLIEST)
print(circuit)

                           ┌──────────┐           ┌──────────┐
(0, 0): ───X^0.1───Z^0.2────@─────────────@──────────────────────────────────────────────────
                            │             │
(0, 1): ───X^0.1───@────────┼─────────────@^0.3────@─────────────────────────────────────────
                   │        │                      │
(0, 2): ───X^0.1───┼────────┼────@─────────────────@^0.3─────────────────────────────────────
                   │        │    │
(1, 0): ───X^0.1───┼────────@^0.3┼────────X────────@─────────────X───X───@───────X───────────
                   │             │                 │                     │
(1, 1): ───X^0.1───@^0.3────@────┼────────X────────┼─────────────────────@^0.3───X───@───────
                            │    │                 │                                 │
(1, 2): ───X^0.1───Z^0.2────┼────@^0.3────X────────┼────@────────X───────────────────@^0.3───
                            │                      │    │
(2, 0): ───X^0.1───X─

In [10]:
## Simulation ##
"""
In Cirq the simulators make a distinction between a 'run' and a 'simulation'.
A run only allows for a simulation that MIMICS the actual quantum hardware. E.g., it
does not allow for access to the amplitudes of the wave function of the system, since
that's not experimentally accessible. To run a simulation of the full circuit we simply
create a simulator, and pass the circuit to the simulator.
"""

simulator = cirq.Simulator()
circuit = cirq.Circuit()
circuit.append(one_step(h, jr, jc, 0.1, 0.2, 0.3))
circuit.append(cirq.measure(*qubits, key='x'))
results = simulator.run(circuit, repetitions=100)
print(results.histogram(key='x'))

Counter({0: 79, 32: 6, 16: 4, 1: 2, 2: 2, 256: 2, 4: 2, 329: 1, 128: 1, 8: 1})


In [11]:
import numpy as np

def energy_func(length, h, jr, jc):
    def energy(measurements):
        # Reshape measurement into array that matches grid shape.
        meas_list_of_lists = [measurements[i * length:(i + 1) * length]
                              for i in range(length)]
        # Convert true/false to +1/-1.
        pm_meas = 1 - 2 * np.array(meas_list_of_lists).astype(np.int32)

        tot_energy = np.sum(pm_meas * h)
        for i, jr_row in enumerate(jr):
            for j, jr_ij in enumerate(jr_row):
                tot_energy += jr_ij * pm_meas[i, j] * pm_meas[i + 1, j]
        for i, jc_row in enumerate(jc):
            for j, jc_ij in enumerate(jc_row):
                tot_energy += jc_ij * pm_meas[i, j] * pm_meas[i, j + 1]
        return tot_energy
    return energy
print(results.histogram(key='x', fold_func=energy_func(3, h, jr, jc)))

Counter({1: 81, 5: 6, -1: 6, -5: 2, -3: 2, 7: 2, -13: 1})


In [12]:
# One can calculate the expectation value over all repetitions.
def obj_func(result):
    energy_hist = result.histogram(key='x', fold_func=energy_func(3, h, jr, jc))
    return np.sum([k * v for k,v in energy_hist.items()]) / result.repetitions
print('Value of the objective function {}'.format(obj_func(results)))

Value of the objective function 0.9


In [13]:
# Parameterizing the ansatz #
"""
On quantum hardware one would most likely want to have
the optimization code as close to the hardware as possible.
"""
import sympy
circuit = cirq.Circuit()
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
circuit.append(one_step(h, jr, jc, alpha, beta, gamma))
circuit.append(cirq.measure(*qubits, key='x'))
print(circuit)

                               ┌──────────────┐             ┌──────────────┐
(0, 0): ───X^alpha───Z^beta─────@─────────────────@────────────────────────────────────────────────────────────M('x')───
                                │                 │                                                            │
(0, 1): ───X^alpha───@──────────┼─────────────────@^gamma────@─────────────────────────────────────────────────M────────
                     │          │                            │                                                 │
(0, 2): ───X^alpha───┼──────────┼──────@─────────────────────@^gamma───────────────────────────────────────────M────────
                     │          │      │                                                                       │
(1, 0): ───X^alpha───┼──────────@^gamma┼──────────X──────────@─────────────────X───X───@─────────X─────────────M────────
                     │                 │                     │                         │            

In [14]:
resolver = cirq.ParamResolver({'alpha': 0.1, 'beta': 0.3, 'gamma': 0.7})
resolved_circuit = cirq.resolve_parameters(circuit, resolver)
# Resolves the parameters to actual values in the above circuit.

In [15]:
"""
A 'sweep' is essentially a collection of parameter resolvers. Sweeps can be created
to specify values directly(this is one way to get classical information into a circuit).
"""
sweep = (cirq.Linspace(key='alpha', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='beta', start=0.1, stop=0.9, length=5)
         * cirq.Linspace(key='gamma', start=0.1, stop=0.9, length=5))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)
for result in results:
    print(result.params.param_dict, obj_func(result))

OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.1)]) 0.94
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.30000000000000004)]) 0.44
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.5)]) 0.82
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.7000000000000001)]) 1.1
OrderedDict([('alpha', 0.1), ('beta', 0.1), ('gamma', 0.9)]) 0.56
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.1)]) 0.74
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.30000000000000004)]) 0.82
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.5)]) 0.74
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.7000000000000001)]) 0.56
OrderedDict([('alpha', 0.1), ('beta', 0.30000000000000004), ('gamma', 0.9)]) 0.56
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.1)]) 0.64
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma', 0.30000000000000004)]) 0.88
OrderedDict([('alpha', 0.1), ('beta', 0.5), ('gamma

OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.5)]) 6.7
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.7000000000000001)]) 6.32
OrderedDict([('alpha', 0.9), ('beta', 0.5), ('gamma', 0.9)]) 6.42
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.1)]) 6.16
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.30000000000000004)]) 6.5
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.5)]) 6.24
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.7000000000000001)]) 6.36
OrderedDict([('alpha', 0.9), ('beta', 0.7000000000000001), ('gamma', 0.9)]) 6.62
OrderedDict([('alpha', 0.9), ('beta', 0.9), ('gamma', 0.1)]) 6.98
OrderedDict([('alpha', 0.9), ('beta', 0.9), ('gamma', 0.30000000000000004)]) 6.7
OrderedDict([('alpha', 0.9), ('beta', 0.9), ('gamma', 0.5)]) 6.62
OrderedDict([('alpha', 0.9), ('beta', 0.9), ('gamma', 0.7000000000000001)]) 6.66
OrderedDict([('alpha', 0.9), ('beta', 0.9), ('gamma', 0.9)]

In [16]:
# Finding the minimum.
sweep_size = 10
sweep = (cirq.Linspace(key='alpha', start=0.0, stop=1.0, length=10)
         * cirq.Linspace(key='beta', start=0.0, stop=1.0, length=10)
         * cirq.Linspace(key='gamma', start=0.0, stop=1.0, length=10))
results = simulator.run_sweep(circuit, params=sweep, repetitions=100)

min = None
min_params = None
for result in results:
    value = obj_func(result)
    if min is None or value < min:
        min = value
        min_params = result.params
print('Minimum objective value is {}.'.format(min))

Minimum objective value is -1.42.


In [17]:
## Circuits ##

# Create a 3x3 grid of qubits using
"""
This is appears like an 8 classical bit represention. If we plug in
qubit[1] we print -> (0, 1), qubit[2] -> (0, 2), qubit[3] -> (1, 0),
qubit[4] -> (1, 1)...etc.
"""
qubits = [cirq.GridQubit(x, y) for x in range(3) for y in range(3)]

print(qubits[0])

(0, 0)


In [18]:
"""
The next concept is that of a 'Gate'. A 'Gate' represents a physical
process that occurs on a Qubit. The important property is that it can
be applied to one or more qubits. We do this via the 'Gate.on' method via '()'
and doing this turns the 'Gate' into a 'GateOperation'.
"""

# This is a Pauli X gate. It is an object instance.
x_gate = cirq.X
# Applying it to the qubit at location (0, 0) (defined above)
# turns it into an operation.
x_op = x_gate(qubits[0])

print(x_op)

X((0, 0))


In [19]:
"""
We've mentioned 'Moment's already, collections of operations.
Here is an example of a Moment in which Pauli X and CZ gate
operate on 3 qubits.
"""

cz = cirq.CZ(qubits[0], qubits[1])
x = cirq.X(qubits[2])
moment = cirq.Moment([x, cz])

print(moment)

X((0, 2)) and CZ((0, 0), (0, 1))


In [20]:
"""
A 'Circuit' is an ordered series of 'Moment's.
Here is a circuit made up of two moments.
"""

cz01 = cirq.CZ(qubits[0], qubits[1])
x2 = cirq.X(qubits[2])
cz12 = cirq.CZ(qubits[1], qubits[2])
moment0 = cirq.Moment([cz01, x2])
moment1 = cirq.Moment([cz12])
circuit = cirq.Circuit((moment0, moment1))

print(circuit)

(0, 0): ───@───────
           │
(0, 1): ───@───@───
               │
(0, 2): ───X───@───


In [21]:
# Construncting the Circuits #
# One of the most useful ways of constructing a circuit is
# by appending onto a 'Circuit' with the 'Circuti.append' method.

from cirq.ops import CZ, H
q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1), H(q2)])

print(circuit)
# this appends an entire new moment to the qubit.

(0, 0): ───@───
           │
(1, 0): ───@───

(2, 0): ───H───


In [22]:
# Let's do it again!
circuit.append([H(q0), CZ(q1, q2)])

print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───@───
               │
(2, 0): ───H───@───


In [23]:
# What happens if we append all of these moments at once.
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1), H(q2), H(q0), CZ(q1, q2)])

print(circuit)
# I don't really understand why appending more gates acting on
# different qubits generates the same two moments?
# What happens to 'CZ(q0, q1)' and 'H(q2)'?? I think this 
# is just a representation of appending both moments at the same time.
# The two examples above appended each moment in sequence.

(0, 0): ───@───H───
           │
(1, 0): ───@───@───
               │
(2, 0): ───H───@───


In [24]:
# InsertStrategies #
"""
'InsertStrategy' defines how Operations are placed in a 'Circuit' when requested
to be inserted at a given 'location'. Here 'location' is defined by the index
of the 'Moment' where the insertion is requested to be plced at. There are 
four different strategies.
"""
# If we first create an 'Operation' in a single moment, and then use 
# 'InsertStrategy.EARLIEST' the 'Operation' can slide back to the 
# first 'Moment' if there is space.

from cirq.circuits import InsertStrategy
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1)])
circuit.append([H(q0), H(q2)], strategy=InsertStrategy.EARLIEST)

print(circuit)
# The H gate at q0 cannot slide back past the CZ gate operators, but
# the H gate at q2 can, and so slides back to end up at the first
# 'Moment'.

(0, 0): ───@───H───
           │
(1, 0): ───@───────

(2, 0): ───H───────


In [25]:
# With 'InsertStrategy.NEW' each operation is created in a new moment.

circuit = cirq.Circuit()
circuit.append([H(q0), H(q1), H(q2)])
print(circuit, "\n")
print("\n")

# Now if apply the method...
circuit = cirq.Circuit()
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.NEW)
print(circuit)
# Useful when you insert a single operation and you don't
# want it to interfere with other 'Moments'.

(0, 0): ───H───

(1, 0): ───H───

(2, 0): ───H─── 



(0, 0): ───H───────────

(1, 0): ───────H───────

(2, 0): ───────────H───


In [26]:
# Inline...
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.INLINE)

print(circuit)

(0, 0): ───────H───────

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───


In [27]:
# Default Strategy...'InsertStrategy.NEW_THEN_INLINE'

circuit = cirq.Circuit()
circuit.append([H(q0)])
circuit.append([CZ(q1,q2), H(q0)], strategy=InsertStrategy.NEW_THEN_INLINE)

print(circuit)

(0, 0): ───H───H───

(1, 0): ───────@───
               │
(2, 0): ───────@───


In [28]:
def my_layer():
    yield CZ(q0, q1)
    yield [H(q) for q in (q0, q1, q2)]
    yield [CZ(q1, q2)]
    yield [H(q0), [CZ(q1, q2)]]
    
circuit = cirq.Circuit()
circuit.append(my_layer())

for x in my_layer():
    print(x)

print("\n")
print(circuit)

CZ((0, 0), (1, 0))
[cirq.H.on(cirq.GridQubit(0, 0)), cirq.H.on(cirq.GridQubit(1, 0)), cirq.H.on(cirq.GridQubit(2, 0))]
[cirq.CZ.on(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]
[cirq.H.on(cirq.GridQubit(0, 0)), [cirq.CZ.on(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]]


(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───


In [29]:
circuit = cirq.Circuit.from_ops(H(q0), H(q1))
print(circuit)

(0, 0): ───H───

(1, 0): ───H───


In [30]:
# Slicing and Iterating over Circuits #
# When circuits are iterated over, each item in the iteration is a moment.

circuit = cirq.Circuit.from_ops(H(q0), CZ(q0, q1))
for moment in circuit:
    print(moment)

H((0, 0))
CZ((0, 0), (1, 0))


In [31]:
circuit = cirq.Circuit.from_ops(H(q0), CZ(q0, q1), H(q1), CZ(q0, q1))
print(circuit[1:3])

print("\n")
# It's useful to have the ability of dropping the last moment,
# which often are just measurements.
print(circuit[:-1])

print("\n")
# Or even reversing a circuit.
print(circuit[::-1])

(0, 0): ───@───────
           │
(1, 0): ───@───H───


(0, 0): ───H───@───────
               │
(1, 0): ───────@───H───


(0, 0): ───@───────@───H───
           │       │
(1, 0): ───@───H───@───────


In [32]:
## Gates ##
# A gate is an operation that can be applied to a collection of
# qubits(objects with Qid). Gates can be applied to qubits by calling
# their 'on' method, or by calling the gate on the qubits.

from cirq.ops import CNOT
from cirq.devices import GridQubit
q0, q1 = (GridQubit(0, 0), GridQubit(0, 1))
print(CNOT.on(q0, q1))
print(CNOT(q0, q1))

CNOT((0, 0), (0, 1))
CNOT((0, 0), (0, 1))


In [33]:
# Gates operate on a specific number of qubits and the classes that implement
# 'Gate' must supply the 'num_qubits' method.

In [34]:
# Magic Mehtods #
# cirq.unitary and def _unitary_
"""
When an object can be described by a unitary matrix, it can expose that matrix
by implementing a '_unitary_(self) -> np.ndarray' method. Callers can query
whether or not an object has a unitary matrix by calling 'cirq.unitary' on it.
"""

# cir.decompose and def _decompose_
"""
Operations and gates can be defined in terms of their operations by using the
'_decompose_' method that returns those other operations.
An example would be 'cirq.CCZ', which decomposes into a series of 'cirq.CNOT' and
'cirq.T' operations. This allows code that doesn't understand 3-qubit operation 
to work with 'cirq.CCZ'; by decomposing it into operations they do understand.

Another example is 'cirq.TOFFOLI', it decomposes into a 'cirq.H' followed by 'cirq.CCZ'
"""

import cirq
print(cirq.unitary(cirq.X))

print("\n")

sqrt_x = cirq.X**0.5
print(cirq.unitary(sqrt_x))
# We can also get values from gates and operations. Like the square root of X gate.

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]


In [35]:
# Gates
"""
Pauli gates included in Cirq use the convention 
'Z**0.5 := S := np.diag(1, i)', 'Z**-0.5 := S**-1',
'X**0.5 := H•S•H', and the square root of 'Y' is infered
via the right hand rule.
"""
# Xmon gates
"""
Google's Xmon devices support a specific gate set. Gates in this gate set operate
on 'GridQubit', which are qubits arranged on a square grid, with x and y coords.

cirq.PhasedXPowGate: This gate is a rotation about an axis in the XY plane of 
the Bloch sphere.
"""
print('\n')

In [36]:
## Simulation ##
# Intro to Pure State Simulation #
# Cirq supports two main types of simulations: 'Pure State' and 'Mixed State'

# Simple Circuit:
import cirq

q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(1, 0)

def basic_circuit(meas=True):
    sqrt_x = cirq.X**0.5
    yield sqrt_x(q0), sqrt_x(q1)
    yield cirq.CZ(q0, q1)
    yield sqrt_x(q0), sqrt_x(q1)
    if meas:
        yield cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')
        
circuit = cirq.Circuit()
circuit.append(basic_circuit())

print(circuit)

(0, 0): ───X^0.5───@───X^0.5───M('q0')───
                   │
(1, 0): ───X^0.5───@───X^0.5───M('q1')───


In [37]:
# Now we can simulate this by creating a 'cirq.Simulator' and
# passing the circuit into its 'run' method.

from cirq import Simulator
simulator = Simulator()
result = simulator.run(circuit)

print(result)

q0=0
q1=1


In [38]:
result = simulator.run(circuit)
print(result)

q0=1
q1=0


In [39]:
import numpy as np
circuit = cirq.Circuit()
circuit.append(basic_circuit(False))
result = simulator.simulate(circuit, qubit_order=[q0, q1])

print(np.around(result.final_state, 3))

[0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]


In [40]:
# Qubit and Amplitude Ordering 

# The 'qubit_order' argument to the simulator's 'run' method determines the ordering of some
# results, like the amplitudes in the final wave function. This is an optional argument.

outside =[1, 10]
inside = [1, 2]
print(np.kron(outside, inside))

[ 1  2 10 20]


In [41]:
i = 0
for first in [0, 1]:
    for second in [0, 1]:
        print('amps[{}] is for first={}, second={}'.format(i , first, second))
        i += 1

amps[0] is for first=0, second=0
amps[1] is for first=0, second=1
amps[2] is for first=1, second=0
amps[3] is for first=1, second=1


In [42]:
# We can check that this is in fact the ordering with a circuit that flips
# one qubit out of two:

q_stay = cirq.NamedQubit('q_stay')
q_flip = cirq.NamedQubit('q_flip')
c = cirq.Circuit.from_ops(cirq.X(q_flip))

# first qubit in order flipped
result = simulator.simulate(c, qubit_order=[q_flip, q_stay])
print(abs(result.final_state).round(3))

# second qubit in order flipped
result = simulator.simulate(c, qubit_order=[q_stay, q_flip])
print(abs(result.final_state).round(3))

[0. 0. 1. 0.]
[0. 1. 0. 0.]


In [43]:
# Stepping through a Circuit #
# It is useful to not only look at the end result of the circuit,
# but also look at or even modify the state of the system at different
# steps in the circuit.

# using an iterator over a Moment by Moment simulation

circuit = cirq.Circuit()
circuit.append(basic_circuit())
for i, step in enumerate(simulator.simulate_moment_steps(circuit)):
    print('state at step %d: %s' % (i, np.around(step.state_vector(), 3)))

state at step 0: [0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
state at step 1: [0. +0.5j 0.5+0.j  0.5+0.j  0. +0.5j]
state at step 2: [0.5+0.j  0. +0.5j 0. +0.5j 0.5+0.j ]
state at step 3: [0.+0.j 0.+1.j 0.+0.j 0.+0.j]


In [44]:
# Monte-Carlo Simulations of noise #
# using a cirq.X Pauli gate we can simulate a
# bitflip and then randomly apply a unitary 
# evolutions with different probabilities

q = cirq.NamedQubit('a')
circuit = cirq.Circuit.from_ops(cirq.bit_flip(p=0.2)(q), cirq.measure(q))
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=100)
print(result.histogram(key='a'))

Counter({0: 76, 1: 24})


In [45]:
# Parameterized Values and Studies #
import sympy
rot_w_gate = cirq.X**sympy.Symbol('x')
circuit  = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
for y in range(5):
    resolver = cirq.ParamResolver({'x': y /4.0})
    result = simulator.simulate(circuit, resolver)
    print(np.round(result.final_state, 2))

[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[ 0.6 +0.6j   0.25-0.25j  0.25-0.25j -0.1 -0.1j ]
[0. +0.5j 0.5+0.j  0.5+0.j  0. -0.5j]
[-0.1 +0.1j   0.25+0.25j  0.25+0.25j  0.6 -0.6j ]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]


In [46]:
# a study ---> a collection of trials

resolvers = [cirq.ParamResolver({'x': y / 2.0}) for y in range(3)]
circuit = cirq.Circuit()
circuit.append([rot_w_gate(q0), rot_w_gate(q1)])
circuit.append([cirq.measure(q0, key='q0'), cirq.measure(q1, key='q1')])
results = simulator.run_sweep(program=circuit,
                              params=resolvers,
                              repetitions=2)

for result in results:
    print(result)

q0=00
q1=00
q0=00
q1=10
q0=11
q1=11


In [47]:
# Mixed state simulations #
# These kinds of simulations are supported
# by the 'cirq.DensityMatrixSimulator' class.
# Here we simulate a single channel using the mixed state sim.

q = cirq.NamedQubit('a')
circuit = cirq.Circuit.from_ops(cirq.H(q), cirq.amplitude_damp(0.2)(q), cirq.measure(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.run(circuit, repetitions=100)
print(result.histogram(key='a'))

Counter({0: 57, 1: 43})


In [48]:
q = cirq.NamedQubit('a')
circuit = cirq.Circuit.from_ops(cirq.H(q), cirq.amplitude_damp(0.2)(q))
simulator = cirq.DensityMatrixSimulator()
result = simulator.simulate(circuit)
print(np.around(result.final_density_matrix, 3))

[[0.6  +0.j 0.447+0.j]
 [0.447+0.j 0.4  +0.j]]


In [49]:
# Schedules and Devices # 
import cirq
from cirq.devices import GridQubit
class Xmon10Device(cirq.Device):
    
    def __init__(self):
        self.qubits = [GridQubit(i, 0) for i in range(10)]
        
    def duration_of(self, operation):
        # Wouldn't it be NOICE if everything took 10ns?!
        return cirq.Duration(nanos=10)
    
    def validate_operation(self, operation):
        if not isinstance(operation, cirq.GateOperation):
            raise ValueError('{!r} is not a supported operation'.format(operation))
        if not isinstance(operation.gate, (cirq.CZPowGate,
                                           cirq.XPowGate,
                                           cirq.PhasedXPowGate,
                                           cirq.YPowGate)):
            raise ValueError('{!r} is not a supported gate'.format(operation.gate))
        if len(operation.qubits) == 2:
            p, q = operation.qubits
            if not p.is_adjacent(q):
                raise ValueError('Non-local interaction: {}'.format(repr(operation)))
                
                
    def validate_scheduled_operation(self, schedule, scheduled_operation):
        self.validate_operation(scheduled_operation.operation)
        
    def validate_circuit(self, circuit):
        for moment in circuit:
            for operation in moment.operations:
                self.validate_operation(operation)
                
    def validate_schedule(self, schedule):
        for scheduled_operation in schedule.scheduled_operations:
            self.validate_scheduled_operation(schedule, scheduled_operation)

In [50]:
# The device that we just created above will
# recognize that two qubit gates between 
# next-nearest-neighbors is not valid.

device = Xmon10Device()
circuit = cirq.Circuit()
circuit.append([cirq.CZ(device.qubits[0], device.qubits[2])])
try:
    device.validate_circuit(circuit)
except ValueError as e:
    print(e)

Non-local interaction: cirq.CZ.on(cirq.GridQubit(0, 0), cirq.GridQubit(2, 0))


In [51]:
# Schedules #
# Schedules contain far more information about timing than 'Moment' structures 
# of a circuit do. This is useful for optimization of a circuit for a 
# particular device.
circuit = cirq.Circuit()
circuit.append([cirq.CZ(device.qubits[0], device.qubits[1]), cirq.X(device.qubits[0])])
print(circuit)

(0, 0): ───@───X───
           │
(1, 0): ───@───────


In [52]:
schedule = cirq.moment_by_moment_schedule(device, circuit)

print(schedule[cirq.Timestamp(nanos=15)])

[ScheduledOperation(cirq.Timestamp(picos=10000), cirq.Duration(picos=10000), cirq.X.on(cirq.GridQubit(0, 0)))]


In [53]:
# slcing notation can be used as well...
slice = schedule[cirq.Timestamp(nanos=5):cirq.Timestamp(nanos=15)]
slice_schedule = cirq.Schedule(device, slice)
print(slice_schedule == schedule)

True


In [71]:
## Qudits ##

# qudits are a generalization to d-dimensional system, qubits are two level quantum systems.

class QutritPlusGate(cirq.SingleQubitGate):
    def _qid_shape_(self):
        return (3,)
    
    def _unitary_(self):
        return np.array([[0, 0, 1],
                         [1, 0, 0],
                         [0, 1, 0]])
    
    def _circuit_diagram_info_(self, args):
        return '[+1]'
    
q0 = cirq.line.line_qubit.LineQid(0, dimension=3)
circuit = cirq.Circuit(
    QutritPlusGate().on(q0)
)

print(circuit)

AttributeError: module 'cirq.line.line_qubit' has no attribute 'LineQid'

In [72]:
### Don't Know What is Going on Above...### 
# making changes to the line_qubit.py file by adding in the LineQid class has not helped.
# There may be a different .py file this program is pulling the modules from.
# Need to run through all of the Cirq files...